<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=beffaf669da4c90302708e241f733213db2c4cfe8c3126d7068f449ab8563eeb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 11:00:47
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 36.45 K (0.26%)
Current PnL: -17.56 L (-11.81%)
CY Booked + Current PnL: -6.53 L (-4.39%)
-------------------
Total profit:  2.07 L
Total loss:  -19.63 L
-------------------
Total Booked + Current PnL: 20.45 L (16.8%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.21 L (54.5%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.09,61.0,H-LC,11.21,173767.0,17590.0,8497.0,-0.38,11.26,4.89,16.71,22.0,2.07,1.33,30.66,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,60.0,M-LC,4.28,217976.0,16496.0,10724.0,0.53,8.19,4.92,13.51,60.0,1.54,1.67,21.82,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,107.26,65.0,M-SC,5.25,89335.0,-11442.0,11533.0,0.79,-11.35,12.91,0.09,245.0,-0.99,0.68,17.37,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.00,72.0,H-MC,3.54,114936.0,14432.0,16206.0,3.70,14.36,14.10,30.49,79.0,0.89,0.88,51.42,MH,ATH,METALS
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203180.0,3042.0,17839.0,0.59,1.52,8.78,10.44,4.0,0.17,1.56,7.17,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226529.0,1567.0,166091.0,1.41,0.70,73.32,74.53,192.0,0.01,1.73,33.17,XY24,BTT,STEEL
4,ANGELONE,3033.00,7.14,38.0,H-SC,6.94,189969.0,-1037.0,58738.0,-0.85,-0.54,30.92,30.21,157.0,-0.02,1.45,20.23,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.11,41.0,H-MC,1.98,135702.0,-86.0,42841.0,-0.30,-0.06,31.57,31.48,107.0,-0.00,1.04,12.05,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245404.0,-2327.0,77376.0,-0.15,-0.94,31.53,30.30,92.0,-0.03,1.88,12.23,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.58,43.0,H-MC,3.22,136656.0,523.0,30597.0,-0.46,0.38,22.39,22.86,91.0,0.02,1.05,13.05,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303146.0,-12696.0,125351.0,-0.14,-4.02,41.35,35.67,5.0,-0.10,2.32,9.28,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.63,217875.0,469.0,53031.0,0.39,0.22,24.34,24.61,48.0,0.01,1.67,55.15,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.37,45.0,H-LC,8.18,172326.0,-9433.0,44684.0,0.15,-5.19,25.93,19.39,86.0,-0.21,1.32,2.54,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.18,229393.0,-12531.0,75791.0,-0.19,-5.18,33.04,26.15,8.0,-0.17,1.76,8.40,X40,ATH,IT
56,RELIANCE,1533.00,-13.66,49.0,H-LC,6.30,215888.0,5522.0,23251.0,0.50,2.62,10.77,13.68,37.0,0.24,1.65,19.58,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.62,55.0,H-LC,1.27,236588.0,-9563.0,88768.0,-0.93,-3.89,37.52,32.18,16.0,-0.11,1.81,30.76,X200,ATH,IT
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203180.0,3042.0,17839.0,0.59,1.52,8.78,10.44,4.0,0.17,1.56,7.17,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,3.33,284825.0,19399.0,37398.0,-0.02,7.31,13.13,21.40,11.0,0.52,2.18,15.66,XY25,NTT,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.94,213446.0,8946.0,32465.0,-0.37,4.37,15.21,20.25,10.0,0.28,1.63,13.49,X40N,BTT,PHARMA
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303146.0,-12696.0,125351.0,-0.14,-4.02,41.35,35.67,5.0,-0.10,2.32,9.28,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,246.25,77.0,M-SC,6.22,154476.0,-28602.0,107577.0,16.04,-15.62,69.64,43.14,230.0,-0.27,1.18,45.31,XY24,NTT,MISC
49,NATIONALUM,247.44,-42.00,72.0,H-MC,3.54,114936.0,14432.0,16206.0,3.70,14.36,14.10,30.49,79.0,0.89,0.88,51.42,MH,ATH,METALS
9,AWL,485.00,-64.78,54.0,H-MC,5.08,216604.0,-51843.0,183529.0,2.46,-19.31,84.73,49.05,116.0,-0.28,1.66,11.37,XY24,NTT,FMCG
57,REPCOHOME,880.00,-60.51,50.0,H-SC,7.96,146802.0,-58196.0,203438.0,2.10,-28.39,138.58,70.85,134.0,-0.29,1.12,20.60,XY24,NTT,FINANCE
36,INDIAMART,4850.92,-50.16,56.0,H-SC,12.20,138982.0,15646.0,118121.0,1.78,12.69,84.99,108.45,119.0,0.13,1.06,37.87,AR,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.0,-38.14,44.0,M-SC,11.97,77044.0,-24115.0,99009.0,-2.18,-23.84,128.51,74.03,202.0,-0.24,0.59,8.64,AR,BTT,AUTO
82,VALIANTORG,838.0,-326.45,55.0,H-SC,8.43,96330.0,-36956.0,121549.0,-1.82,-27.73,126.18,63.47,139.0,-0.30,0.74,61.47,XR,NTT,CHEMICALS
78,UJJIVANSFB,60.0,103.03,66.0,H-SC,12.71,128547.0,-13932.0,33448.0,-1.53,-9.78,26.02,13.70,163.0,-0.42,0.98,51.72,OX40N,NTT,BANKS
81,VAIBHAVGBL,521.0,53.17,61.0,H-SC,10.20,102046.0,-49945.0,124068.0,-1.45,-32.86,121.58,48.77,125.0,-0.40,0.78,18.78,XR,NTT,JEWELLERY
39,INFY,2275.0,-17.44,54.0,H-LC,9.80,322099.0,9211.0,162467.0,-1.33,2.94,50.44,54.87,3.0,0.06,2.47,9.79,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,66.0,H-SC,12.71,128547.0,-13932.0,33448.0,-1.53,-9.78,26.02,13.70,163.0,-0.42,0.98,51.72,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,107.26,65.0,M-SC,5.25,89335.0,-11442.0,11533.0,0.79,-11.35,12.91,0.09,245.0,-0.99,0.68,17.37,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-68.16,54.0,H-SC,2.33,219717.0,-49950.0,86283.0,-0.07,-18.52,39.27,13.47,138.0,-0.58,1.68,11.13,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,115.40,51.0,M-SC,2.75,141360.0,-33199.0,33234.0,0.74,-19.02,23.51,0.02,221.0,-1.00,1.08,22.96,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.23,114039.0,-29764.0,56506.0,-0.60,-20.70,49.55,18.60,149.0,-0.53,0.87,25.85,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,52.0,H-LC,13.69,292584.0,-53372.0,177218.0,0.08,-15.43,60.57,35.80,1.0,-0.30,2.24,3.65,X40,BTT,IT
39,INFY,2275.00,-17.44,54.0,H-LC,9.80,322099.0,9211.0,162467.0,-1.33,2.94,50.44,54.87,3.0,0.06,2.47,9.79,X40,BTT,IT
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203180.0,3042.0,17839.0,0.59,1.52,8.78,10.44,4.0,0.17,1.56,7.17,X40,NTT,FMCG
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303146.0,-12696.0,125351.0,-0.14,-4.02,41.35,35.67,5.0,-0.10,2.32,9.28,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,52.0,H-LC,12.29,249768.0,-11851.0,131053.0,0.81,-4.53,52.47,45.57,7.0,-0.09,1.91,6.28,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226529.0,1567.0,166091.0,1.41,0.70,73.32,74.53,192.0,0.01,1.73,33.17,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.74,188540.0,23870.0,44100.0,-0.23,14.50,23.39,41.28,89.0,0.54,1.44,23.64,X40N,ATH,AC
84,VOLTAS,1918.49,-1.50,51.0,H-MC,3.18,206700.0,14958.0,81068.0,-1.21,7.80,39.22,50.08,99.0,0.18,1.58,15.13,XY25,ATH,AC
24,DABUR,735.00,-2.58,60.0,H-MC,6.00,212043.0,15015.0,74597.0,-0.69,7.62,35.18,45.49,102.0,0.20,1.62,21.65,XY24,BTT,FMCG
18,CAMS,5211.76,-4.05,55.0,H-SC,4.75,109172.0,7168.0,36758.0,0.20,7.03,33.67,43.06,122.0,0.20,0.84,25.35,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.00,72.0,H-MC,3.54,114936.0,14432.0,16206.0,3.70,14.36,14.10,30.49,79.0,0.89,0.88,51.42,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.28,62.0,M-SC,1.84,102585.0,11086.0,67665.0,-0.15,12.12,65.96,86.07,223.0,0.16,0.79,48.97,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.16,56.0,H-SC,12.20,138982.0,15646.0,118121.0,1.78,12.69,84.99,108.45,119.0,0.13,1.06,37.87,AR,ATH,MISC
86,WIPRO,420.00,-10.62,55.0,M-LC,6.28,156798.0,5853.0,103596.0,-1.11,3.88,66.07,72.51,53.0,0.06,1.20,10.13,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.53,183483.0,3675.0,108732.0,1.54,2.04,59.26,62.52,88.0,0.03,1.40,36.83,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6733.33,52.0,L-SC,5.97,74951.0,-18859.0,95480.0,0.37,-20.10,127.39,81.67,269.0,-0.20,0.57,45.15,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.00,47.0,H-SC,14.34,89393.0,-11570.0,106101.0,0.49,-11.46,118.69,93.63,148.0,-0.11,0.68,33.15,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,58.0,L-SC,43.46,89088.0,-24461.0,64509.0,-0.54,-21.54,72.41,35.27,268.0,-0.38,0.68,118.05,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.07,95100.0,-2880.0,24479.0,0.83,-2.94,25.74,22.05,152.0,-0.12,0.73,38.23,XR,ATH,FINANCE
82,VALIANTORG,838.00,-326.45,55.0,H-SC,8.43,96330.0,-36956.0,121549.0,-1.82,-27.73,126.18,63.47,139.0,-0.30,0.74,61.47,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,58.0,L-SC,43.46,89088.0,-24461.0,64509.0,-0.54,-21.54,72.41,35.27,268.0,-0.38,0.68,118.05,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,3.01,82386.0,-19824.0,113215.0,0.42,-19.40,137.42,91.37,208.0,-0.18,0.63,33.74,XY25,NTT,FINANCE
75,TITAGARH,1548.00,-3.20,64.0,H-SC,6.48,158638.0,-33377.0,112268.0,-0.26,-17.38,70.77,41.08,158.0,-0.30,1.21,34.43,XY24,NTT,ENGINEERING
49,NATIONALUM,247.44,-42.00,72.0,H-MC,3.54,114936.0,14432.0,16206.0,3.70,14.36,14.10,30.49,79.0,0.89,0.88,51.42,MH,ATH,METALS
8,ATULAUTO,844.00,3629.73,59.0,M-SC,7.46,116254.0,-31114.0,82936.0,-0.78,-21.11,71.34,35.16,236.0,-0.38,0.89,18.74,XY24,NTT,AUTO


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.27
1,25,44.34
2,50,75.12


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.85
LC    35.37
MC    23.72
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.81
X40      16.04
X40N     13.06
XY25     10.25
XR        9.51
AR        8.33
OX40N     7.73
MH        1.82
X200      1.81
X5K       1.46
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.79
H-SC    24.90
H-MC    20.63
M-SC    14.30
M-LC     7.47
M-MC     2.76
L-SC     1.65
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.89,-15.84,74.49
FMCG,12.30,-3.34,38.37
FINANCE,9.05,-18.02,60.46
BANKS,7.81,-11.70,59.14
MISC,6.21,-13.32,78.27
PAINTS,6.15,-14.88,40.59
ELECTRICAL,5.85,-7.61,66.91
AC,3.77,1.72,31.56
AUTO,3.45,-16.66,69.27


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2696917.0,22
AR,1179740.0,9
XR,1052158.0,13
X40,875336.0,10
X40N,567697.0,10
OX40N,447094.0,10
XY25,411855.0,7
SR,196484.0,2
X5K,126324.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2766677.0,28
M-SC,1377345.0,17
H-LC,1366066.0,15
H-MC,1148584.0,15
M-LC,402644.0,5
M-MC,316105.0,2
L-SC,244427.0,3
L-MC,59750.0,1
L-LC,39080.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,912831.0,6
M-SC,XY24,794990.0,7
H-SC,AR,648942.0,5
H-LC,X40,582769.0,5
H-SC,XR,505447.0,6
H-MC,XY24,457960.0,4
H-LC,AR,369852.0,2
M-MC,XY24,316105.0,2
H-SC,X40N,247577.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
